In [33]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [34]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [35]:
feedback_bits = 256
ncodebooks=128
ncentroids=256
train_sam_num = 3000 # 训练集样本数
split_Br = 2 # A的列、B的行分割成split_br份
split_Bc = 2 # B的列分割成split_bc份
amm_ind_list = [(0, 0), (1, 0), (1, 1)]#[(0,0), (0, 1), (1, 0), (1, 1)] # 使用近似矩阵乘的小矩阵索引（针对矩阵B）
amm_simple_ind_list = []
for ind in amm_ind_list:
    amm_simple_ind_list.append(ind[0]*split_Bc + ind[1])

In [36]:
# zx = np.random.randint(1,100,(11,3))
# print(zx)
# mins_new, ranges_new = saturation_sort(zx)
# print(mins_new, ranges_new)

In [37]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")

dir_train_split = os.path.join(dir_train, 'train', 'f'+str(feedback_bits), 'split_Br'+str(split_Br)+'_split_Bc'+str(split_Bc))
try:
    os.mkdir(dir_train_split)
except FileExistsError:
    pass
# print(dir_train_split)
dir_test_split = os.path.join(dir_train, 'test', 'f'+str(feedback_bits), 'split_Br'+str(split_Br)+'_split_Bc'+str(split_Bc))
try:
    os.mkdir(dir_test_split)
except FileExistsError:
    pass 


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'
dir_result = os.path.join(dir_result, method)
# print(dir_result)


### 切分训练集

切分训练集A矩阵

In [38]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
split_size_Ac = int(data_to_fc_train.shape[1] / split_Br) # split后单个训练集A的列数
data_to_fc_train_split_path_list = []
for i in range(split_Br):
    data_to_fc_train_split_path_list.append(os.path.join(dir_train_split, 'data_to_fc1_train_f'+str(feedback_bits)+'_split'+str(split_Br)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_train_split_path_list[i], data_to_fc_train[np.ix_(range(data_to_fc_train.shape[0]), range(i*split_size_Ac,(i+1)*split_size_Ac))])

(96000, 2048)


切分训练集B矩阵

In [39]:
weight_train = np.load(os.path.join(dir_train, weightpath))
print(weight_train.shape)
split_size_Br = split_size_Ac # split后 B的行数
split_size_Bc = int(weight_train.shape[1] / split_Bc) # split后B的列数
split_total_num_B = split_size_Bc * split_size_Br # B矩阵分成了多少块
weight_train_split_path_list = []
B_split_ind = 0 # B的分块的序号，从0开始
for i in range(split_Br):
    for j in range(split_Bc):
        weight_train_split_path_list.append(os.path.join(dir_train_split, 'weight_fc1_train_f'+str(feedback_bits)+'_sr'+str(split_Br)+'_'+str(i)+'_sc'+str(split_Bc)+'_'+str(j)+'.npy'))
        np.save(weight_train_split_path_list[i*split_Bc + j], weight_train[np.ix_(range(i*split_size_Br,(i+1)*split_size_Br), range(j*split_size_Bc,(j+1)*split_size_Bc))])
        

(2048, 256)


训练集A与B相乘

In [40]:
y_train_split_path_list = []
for i in range(split_Br):
    for j in range(split_Bc):
        y_train_split_path_list.append(os.path.join(dir_train_split, 'y_fc1_train_f'+str(feedback_bits)+'_sr'+str(split_Br)+'_'+str(i)+'_sc'+str(split_Bc)+'_'+str(j)+'.npy'))
        xx = np.load(data_to_fc_train_split_path_list[i])
        ww = np.load(weight_train_split_path_list[i*split_Bc + j])
        np.save(y_train_split_path_list[i*split_Bc + j], np.matmul(xx, ww))

### 切分测试集

In [41]:
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
# split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_test_split_path_list = []
for i in range(split_Br):
    data_to_fc_test_split_path_list.append(os.path.join(dir_test_split, 'data_to_fc1_test_f'+str(feedback_bits)+'_split'+str(split_Br)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_test_split_path_list[i], data_to_fc_test[np.ix_(range(data_to_fc_test.shape[0]), range(i*split_size_Ac,(i+1)*split_size_Ac))])

(32000, 2048)


In [42]:
# # 输入与weight相乘
# y_test_split_path_list = []
# for i in range(split):
#     y_test_split_path_list.append(os.path.join(dir_test_split, 'y_fc1_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
#     xx = np.load(data_to_fc_test_split_path_list[i])
#     ww = np.load(weight_train_split_path_list[i])
#     np.save(y_test_split_path_list[i], np.matmul(xx, ww))

### AMM训练

In [43]:
est_list = []
for i in range(split_Br):
    for j in range(split_Bc):
        if (i, j) in amm_ind_list:
            dir_est, X_path = os.path.split(data_to_fc_train_split_path_list[i])
            dir_est, W_path = os.path.split(weight_train_split_path_list[i*split_Bc + j])
            dir_est, Y_path = os.path.split(y_train_split_path_list[i*split_Bc + j])
            est3 = mm.estFactory(X_path=X_path, W_path=W_path, Y_path=Y_path, dir= dir_est, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])
            est_list.append(est3)



# for i in range(split):
#     dir_est, X_path = os.path.split(data_to_fc_train_split_path_list[i])
#     dir_est, W_path = os.path.split(weight_train_split_path_list[i])
#     dir_est, Y_path = os.path.split(y_train_split_path_list[i])
#     est3 = mm.estFactory(X_path=X_path, W_path=W_path, Y_path=Y_path, dir= dir_est, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])
#     est_list.append(est3)


running method:  PQ
running kmeans in subspace 1/128... mse / {var(X_subs), var(X)}: 0.127, 0.0139
running kmeans in subspace 2/128... mse / {var(X_subs), var(X)}: 0.109, 0.134
running kmeans in subspace 3/128... mse / {var(X_subs), var(X)}: 0.141, 0.0251
running kmeans in subspace 4/128... mse / {var(X_subs), var(X)}: 0.153, 0.0102
running kmeans in subspace 5/128... mse / {var(X_subs), var(X)}: 0.104, 0.00873
running kmeans in subspace 6/128... mse / {var(X_subs), var(X)}: 0.135, 0.0152
running kmeans in subspace 7/128... mse / {var(X_subs), var(X)}: 0.156, 0.00713
running kmeans in subspace 8/128... mse / {var(X_subs), var(X)}: 0.16, 0.00817
running kmeans in subspace 9/128... mse / {var(X_subs), var(X)}: 0.103, 0.029
running kmeans in subspace 10/128... mse / {var(X_subs), var(X)}: 0.118, 0.309
running kmeans in subspace 11/128... mse / {var(X_subs), var(X)}: 0.188, 0.0416
running kmeans in subspace 12/128... mse / {var(X_subs), var(X)}: 0.197, 0.0175
running kmeans in subspace 13/

In [44]:
y_split_out_matmul_list = []
i_est_list = 0 # est_list的索引
for i in range(split_Br):
    for j in range(split_Bc):
        x_test = np.load(data_to_fc_test_split_path_list[i])
        w_test = np.load(weight_train_split_path_list[i*split_Bc + j])
        if (i, j) in amm_ind_list: # 用近似矩阵乘法的分块
            y_split_out_matmul = mm.eval_matmul(est_list[i_est_list], x_test, w_test)
            i_est_list += 1
        else:
            y_split_out_matmul = np.matmul(x_test, w_test)
        y_split_out_matmul_list.append(y_split_out_matmul)

In [45]:
# split 的矩阵结果合成
# 先纵向相加
sum_col_list = []
for j in range(split_Bc):
    sum_col = np.zeros((y_split_out_matmul_list[0].shape[0], split_size_Bc))
    for i in range(split_Br):
        sum_col += y_split_out_matmul_list[i * split_Bc + j]
    sum_col_list.append(sum_col)

#再横向拼接
y_out_matmul = sum_col_list[0]
if split_Bc > 1:
    for j in range(1, split_Bc):
        y_out_matmul = np.append(y_out_matmul, sum_col_list[j], axis=1)


In [46]:
bias = np.load(dir_train+'/'+biaspath)
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [47]:
amm_simple_ind_str= ''
for ind in amm_simple_ind_list:
    amm_simple_ind_str += str(ind)

# print(amm_simple_ind_str)

In [48]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)

np.save(dir_result+'/'+method+'fc1_sr%i_sc%i_amm%s_fb%i_cb%i_ct%i.npy' % (split_Br, split_Bc, amm_simple_ind_str, feedback_bits, ncodebooks, ncentroids), y_out_last.astype(np.float32))
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.11847195 -0.09605301 -0.51238856 ... -0.16710116 -0.04142496
  -0.11754728]
 [-0.30603447 -0.19961322 -0.39775324 ... -0.42563039 -0.04886958
  -0.17213564]
 [ 0.19138459 -0.08196624 -0.39668293 ... -0.44191162  0.17071761
   0.1273084 ]
 ...
 [ 0.07198246 -0.06401942 -0.74603733 ... -0.16405596 -0.29358522
  -0.078456  ]
 [ 0.18419061 -0.03619562 -0.28335591 ...  0.07578072 -0.08918835
  -0.10354177]
 [-0.44284906 -0.05619076 -0.58925023 ... -0.35032915 -0.28054313
   0.07623202]]
(32000, 256)
